# Semi-Supervised Architope (Financial Data)
---
- This code Implements Algorithm 3.2 of the "Architopes" paper.

#### Mode: Code-Testin Parameter(s)

In [1]:
# trial_run = True

### Meta-parameters

In [2]:
# Test-size Ratio
test_size_ratio = 0.90
min_height = 50

### Hyperparameters

Only turn of if running code directly here, typically this script should be run be called by other notebooks.  

In [3]:
# load dataset
results_path = "./outputs/models/"
results_tables_path = "./outputs/results/"
raw_data_path_folder = "./inputs/raw/"
data_path_folder = "./inputs/data/"

### Import

In [4]:
# Load Packages/Modules
exec(open('Init_Dump.py').read())
# Load Hyper-parameter Grid
exec(open('Grid_Enhanced_Network.py').read())
# Load Helper Function(s)
exec(open('Helper_Functions.py').read())
# Pre-process Data
exec(open('Financial_Data_Preprocessor.py').read())
# Import time separately
import time

Using TensorFlow backend.


Deep Feature Builder - Ready
Deep Classifier - Ready
#================================================#
 Training Datasize: 539 and test datasize: 60.  
#================================================#


### Set Seed

In [5]:
random.seed(2021)
tf.random.set_seed(2021)

#### Pre-Process:
- Convert Categorical Variables to Dummies
- Remove Bad Column
- Perform Training/Test Split

# Random Lipschitz Partition Builder

We implement the random paritioning method of [Yair Bartal](https://scholar.google.com/citations?user=eCXP24kAAAAJ&hl=en):
- [On approximating arbitrary metrices by tree metrics](https://dl.acm.org/doi/10.1145/276698.276725)

The algorithm is summarized as follow:

---

## Algorithm:
 1. Sample $\alpha \in [4^{-1},2^{-1}]$ randomly and uniformly,
 2. Apply a random suffle of the data, (a random bijection $\pi:\{i\}_{i=1}^X \rightarrow \mathbb{X}$),
 3. For $i = 1,\dots,I$:
   - Set $K_i\triangleq B\left(\pi(i),\alpha \Delta \right) - \bigcup_{j=1}^{i-1} P_j$
 
 4. Remove empty members of $\left\{K_i\right\}_{i=1}^X$.  
 
 **Return**: $\left\{K_i\right\}_{i=1}^{\tilde{X}}$.  
 
 For more details on the random-Lipschitz partition of Yair Bartal, see this [well-written blog post](https://nickhar.wordpress.com/2012/03/26/lecture-22-random-partitions-of-metric-spaces/).

## Define Random Partition Builder

In [6]:
from scipy.spatial import distance_matrix

Here we use $\Delta_{in} = Q_{q}\left(\Delta(\mathbb{X})\right)$ where $\Delta(\mathbb{X})$ is the vector of (Euclidean) distances between the given data-points, $q \in (0,1)$ is a hyper-parameter, and $Q$ is the empirical quantile function.

In [7]:
def Random_Lipschitz_Partioner(Min_data_size_percentage,q_in, X_train_in,y_train_in, CV_folds_failsafe, min_size):
       
    #-----------------------#
    # Reset Seed Internally #
    #-----------------------#
    random.seed(2020)
    np.random.seed(2020)

    #-------------------------------------------#
    #-------------------------------------------#
    # 1) Sample radius from unifom distribution #
    #-------------------------------------------#
    #-------------------------------------------#
    alpha = np.random.uniform(low=.25,high=.5,size=1)[0]

    #-------------------------------------#
    #-------------------------------------#
    # 2) Apply Random Bijection (Shuffle) #
    #-------------------------------------#
    #-------------------------------------#
    X_train_in_shuffled = X_train_in#.sample(frac=1)
    y_train_in_shuffled = y_train_in#.sample(frac=1)

    #--------------------#
    #--------------------#
    # X) Initializations #
    #--------------------#
    #--------------------#
    # Compute-data-driven radius
    Delta_X = distance_matrix(X_train_in_shuffled,X_train_in_shuffled)[::,0]
    Delta_in = np.quantile(Delta_X,q_in)

    # Initialize Random Radius
    rand_radius = Delta_in*alpha

    # Initialize Data_sizes & ratios
    N_tot = X_train_in.shape[0] #<- Total number of data-points in input data-set!
    N_radios = np.array([])
    N_pool_train_loop = N_tot
    # Initialize List of Dataframes
    X_internal_train_list = list()
    y_internal_train_list = list()

    # Initialize Partioned Data-pool
    X_internal_train_pool = X_train_in_shuffled
    y_internal_train_pool = y_train_in_shuffled

    # Initialize counter 
    part_current_loop = 0

    #----------------------------#
    #----------------------------#
    # 3) Iteratively Build Parts #
    #----------------------------#
    #----------------------------#

    while ((N_pool_train_loop/N_tot > Min_data_size_percentage) or (X_internal_train_pool.empty == False)):
        # Extract Current Center
        center_loop = X_internal_train_pool.iloc[0]
        # Compute Distances
        ## Training
        distances_pool_loop_train = X_internal_train_pool.sub(center_loop)
        distances_pool_loop_train = np.array(np.sqrt(np.square(distances_pool_loop_train).sum(axis=1)))
        # Evaluate which Distances are less than the given random radius
        Part_train_loop = X_internal_train_pool[distances_pool_loop_train<rand_radius]
        Part_train_loop_y = y_internal_train_pool[distances_pool_loop_train<rand_radius]

        # Remove all data-points which are "too small"
        if X_internal_train_pool.shape[0] > max(CV_folds,4):
            # Append Current part to list
            X_internal_train_list.append(Part_train_loop)
            y_internal_train_list.append(Part_train_loop_y)

        # Remove current part from pool 
        X_internal_train_pool = X_internal_train_pool[(np.logical_not(distances_pool_loop_train<rand_radius))]
        y_internal_train_pool = y_internal_train_pool[(np.logical_not(distances_pool_loop_train<rand_radius))]

        # Update Current size of pool of training data
        N_pool_train_loop = X_internal_train_pool.shape[0]
        N_radios = np.append(N_radios,(N_pool_train_loop/N_tot))

        # Update Counter
        part_current_loop = part_current_loop +1
        
        # Update User
        print((N_pool_train_loop/N_tot))


    # Post processing #
    #-----------------#
    # Remove Empty Partitions
    N_radios = N_radios[N_radios>0]
    
    
    #-----------------------------------------------------------------#
    # Combine parts which are too small to perform CV without an error
    #-----------------------------------------------------------------#
    # Initialize lists (partitions) with "enough" datums per part
    X_internal_train_list_good = list()
    y_internal_train_list_good = list()
    X_small_parts = list()
    y_small_parts = list()
    # Initialize first list item test
    is_first = True
    # Initialize counter
    goods_counter = 0
    for search_i in range(len(X_internal_train_list)):
        number_of_instances_in_part = len(X_internal_train_list[search_i]) 
        if number_of_instances_in_part < max(CV_folds_failsafe,min_size):
            # Check if first 
            if is_first:
                # Initialize set of small X_parts
                X_small_parts = X_internal_train_list[search_i]
                # Initialize set of small y_parts
                y_small_parts = y_internal_train_list[search_i]

                # Set is_first to false
                is_first = False
            else:
                X_small_parts = X_small_parts.append(X_internal_train_list[search_i])
                y_small_parts = np.append(y_small_parts,y_internal_train_list[search_i])
#                 y_small_parts = y_small_parts.append(y_internal_train_list[search_i])
        else:
            # Append to current list
            X_internal_train_list_good.append(X_internal_train_list[search_i])
            y_internal_train_list_good.append(y_internal_train_list[search_i])
            # Update goods counter 
            goods_counter = goods_counter +1

    # Append final one to good list
    X_internal_train_list_good.append(X_small_parts)
    y_internal_train_list_good.append(y_small_parts)

    # reset is_first to false (inscase we want to re-run this particular block)
    is_first = True

    # Set good lists to regular lists
    X_internal_train_list = X_internal_train_list_good
    y_internal_train_list = y_internal_train_list_good
    
    
    
    # Return Value #
    #--------------#
    return [X_internal_train_list, y_internal_train_list, N_radios]

# Apply Random Partitioner to the given Dataset

In [8]:
import time
partitioning_time_begin = time.time()

In [9]:
if Option_Function == 'SnP':
    q_in_auto = .8
    Min_data_size_percentage_auto = .1
else:
    if Option_Function == 'crypto':
        q_in_auto = .99
        Min_data_size_percentage_auto = .3
    else:
        q_in_auto = .5
        Min_data_size_percentage_auto = .3

In [10]:
# Initialize Number of Parts currently generated
N_parts_generated = 0

# Generate Partition (with option to regernerate if only 1 part is randomly produced)
while N_parts_generated < 2:
    # Generate Parts
    X_parts_list, y_parts_list, N_ratios = Random_Lipschitz_Partioner(Min_data_size_percentage=Min_data_size_percentage_auto, 
                                                                      q_in=q_in_auto, 
                                                                      X_train_in=X_train, 
                                                                      y_train_in=data_y, 
                                                                      CV_folds_failsafe=CV_folds,
                                                                      min_size = 100)
    
    # Update Number of Parts
    N_parts_generated = len(X_parts_list)
    # Shuffle hyperparameters
    Min_data_size_percentage_auto = (Min_data_size_percentage_auto + random.uniform(0,.3)) % 1
    q_in_auto = (q_in_auto + random.uniform(0,.3)) % 1
    
    # Update User
    print('The_parts_listhe number of parts are: ' + str(len(X_parts_list))+'.')

0.8200371057513914
0.8181818181818182
0.7606679035250464
0.7588126159554731
0.7569573283858998
0.75139146567718
0.7365491651205937
0.725417439703154
0.7235621521335807
0.7198515769944341
0.6846011131725418
0.6827458256029685
0.6808905380333952
0.6790352504638218
0.6697588126159555
0.6623376623376623
0.6586270871985158
0.647495361781076
0.6419294990723562
0.6307977736549165
0.6196660482374768
0.6066790352504638
0.5974025974025974
0.5862708719851577
0.5844155844155844
0.5732838589981447
0.5695732838589982
0.5528756957328386
0.5510204081632653
0.549165120593692
0.5454545454545454
0.5435992578849722
0.5417439703153989
0.5398886827458256
0.5380333951762524
0.536178107606679
0.5269016697588126
0.5213358070500927
0.5194805194805194
0.5176252319109462
0.5157699443413729
0.5120593692022264
0.5064935064935064
0.5027829313543599
0.49907235621521334
0.49536178107606677
0.4935064935064935
0.4897959183673469
0.48794063079777367
0.48608534322820035
0.48237476808905383
0.4805194805194805
0.47866419294

0.07421150278293136
0.07235621521335807
0.07050092764378478
0.0686456400742115
0.06679035250463822
0.06493506493506493
0.06307977736549165
0.061224489795918366
0.059369202226345084
0.0575139146567718
0.055658627087198514
0.05380333951762523
0.05194805194805195
0.05009276437847866
0.04823747680890538
0.04638218923933209
0.04452690166975881
0.04267161410018553
0.04081632653061224
0.03896103896103896
0.03710575139146568
0.03525046382189239
0.03339517625231911
0.03153988868274583
0.029684601113172542
0.027829313543599257
0.025974025974025976
0.02411873840445269
0.022263450834879406
0.02040816326530612
0.01855287569573284
0.016697588126159554
0.014842300556586271
0.012987012987012988
0.011131725417439703
0.00927643784786642
0.0074211502782931356
0.0055658627087198514
0.0037105751391465678
0.0018552875695732839
0.0
The_parts_listhe number of parts are: 2.


In [11]:
partitioning_time = time.time() - partitioning_time_begin

In [12]:
print('The_parts_listhe number of parts are: ' + str(len(X_parts_list))+'.')

The_parts_listhe number of parts are: 2.


#### Building Training Predictions on each part
- Train locally (on each "naive part")
- Generate predictions for (full) training and testings sets respectively, to be used in training the classifer and for prediction, respectively.  
- Generate predictions on all of testing-set (will be selected between later using classifier)

In [13]:
# Time-Elapse (Start) for Training on Each Part
Architope_partition_training_begin = time.time()
# Initialize running max for Parallel time
Architope_partitioning_max_time_running = -math.inf # Initialize slowest-time at - infinity to force updating!
# Initialize N_parameter counter for Architope
N_params_Architope = 0

In [14]:
for current_part in range(len(X_parts_list)):
    #==============#
    # Timer(begin) #
    #==============#
    current_part_training_time_for_parallel_begin = time.time()
    
    
    # Initializations #
    #-----------------#
    # Reload Grid
    exec(open('Grid_Enhanced_Network.py').read())
    # Modify heights according to optimal (data-driven) rule (with threshold)
    current_height = np.ceil(np.array(param_grid_Vanilla_Nets['height'])*N_ratios[current_part])
    current_height_threshold = np.repeat(min_height,(current_height.shape[0]))
    current_height = np.maximum(current_height,current_height_threshold)
    current_height = current_height.astype(int).tolist()
    param_grid_Vanilla_Nets['height'] = current_height
    # Automatically Fix Input Dimension
    param_grid_Vanilla_Nets['input_dim'] = [X_train.shape[1]]
    param_grid_Vanilla_Nets['output_dim'] = [1]
    
    # Update User #
    #-------------#
    print('Status: Current part: ' + str(current_part) + ' out of : '+str(len(X_parts_list)) +' parts.')
    print('Heights to iterate over: '+str(current_height))
    
    # Generate Prediction(s) on current Part #
    #----------------------------------------#
    # Failsafe (number of data-points)
    CV_folds_failsafe = min(CV_folds,max(1,(X_train.shape[0]-1)))
    # Train Network
    y_hat_train_full_loop, y_hat_test_full_loop, N_params_Architope_loop = build_ffNN(n_folds = CV_folds_failsafe, 
                                                                                     n_jobs = n_jobs,
                                                                                     n_iter = n_iter, 
                                                                                     param_grid_in = param_grid_Vanilla_Nets, 
                                                                                     X_train= X_parts_list[current_part], 
                                                                                     y_train=y_parts_list[current_part],
                                                                                     X_test_partial=X_train,
                                                                                     X_test=X_test)
    
    # Append predictions to data-frames
    ## If first prediction we initialize data-frames
    if current_part==0:
        # Register quality
        training_quality = np.array(np.abs(y_hat_train_full_loop-y_train))
        training_quality = training_quality.reshape(training_quality.shape[0],1)

        # Save Predictions
        predictions_train = y_hat_train_full_loop
        predictions_train = predictions_train.reshape(predictions_train.shape[0],1)
        predictions_test = y_hat_test_full_loop
        predictions_test = predictions_test.reshape(predictions_test.shape[0],1)
        
        
    ## If not first prediction we append to already initialized dataframes
    else:
    # Register Best Scores
        #----------------------#
        # Write Predictions 
        # Save Predictions
        y_hat_train_loop = y_hat_train_full_loop.reshape(predictions_train.shape[0],1)
        predictions_train = np.append(predictions_train,y_hat_train_loop,axis=1)
        y_hat_test_loop = y_hat_test_full_loop.reshape(predictions_test.shape[0],1)
        predictions_test = np.append(predictions_test,y_hat_test_loop,axis=1)
        
        # Evaluate Errors #
        #-----------------#
        # Training
        prediction_errors = np.abs(y_hat_train_loop.reshape(-1,)-y_train)
        training_quality = np.append(training_quality,prediction_errors.reshape(training_quality.shape[0],1),axis=1)
        
    #============#
    # Timer(end) #
    #============#
    current_part_training_time_for_parallel = time.time() - current_part_training_time_for_parallel_begin
    Architope_partitioning_max_time_running = max(Architope_partitioning_max_time_running,current_part_training_time_for_parallel)

    #============---===============#
    # N_parameter Counter (Update) #
    #------------===---------------#
    N_params_Architope = N_params_Architope + N_params_Architope_loop

# Update User
#-------------#
print(' ')
print(' ')
print(' ')
print('----------------------------------------------------')
print('Feature Generation (Learning Phase): Score Generated')
print('----------------------------------------------------')
print(' ')
print(' ')
print(' ')

Status: Current part: 0 out of : 2 parts.
Heights to iterate over: [50]
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    7.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    7.3s finished


Epoch 1/200
11/11 [==============================] - 0s 1ms/step - loss: 9.1937 - mse: 139.7442 - mae: 9.1937 - mape: 99.5414
Epoch 2/200
11/11 [==============================] - 0s 2ms/step - loss: 7.3139 - mse: 88.3630 - mae: 7.3139 - mape: 105.1338
Epoch 3/200
11/11 [==============================] - 0s 3ms/step - loss: 3.3153 - mse: 18.9093 - mae: 3.3153 - mape: 142.3476
Epoch 4/200
11/11 [==============================] - 0s 2ms/step - loss: 2.7150 - mse: 12.6905 - mae: 2.7150 - mape: 129.1352
Epoch 5/200
11/11 [==============================] - 0s 2ms/step - loss: 2.2800 - mse: 9.2129 - mae: 2.2800 - mape: 96.8482
Epoch 6/200
11/11 [==============================] - 0s 2ms/step - loss: 1.6574 - mse: 4.8894 - mae: 1.6574 - mape: 82.6039
Epoch 7/200
11/11 [==============================] - 0s 2ms/step - loss: 1.4920 - mse: 3.9936 - mae: 1.4920 - mape: 68.0452
Epoch 8/200
11/11 [==============================] - 0s 3ms/step - loss: 1.2350 - mse: 2.9614 - mae: 1.2350 - mape: 51.7620


11/11 [==============================] - 0s 2ms/step - loss: 0.3454 - mse: 0.2446 - mae: 0.3454 - mape: 9.5148
Epoch 67/200
11/11 [==============================] - 0s 2ms/step - loss: 0.2681 - mse: 0.1327 - mae: 0.2681 - mape: 8.5998
Epoch 68/200
11/11 [==============================] - 0s 2ms/step - loss: 0.3075 - mse: 0.1529 - mae: 0.3075 - mape: 13.9241
Epoch 69/200
11/11 [==============================] - 0s 2ms/step - loss: 0.2556 - mse: 0.1122 - mae: 0.2556 - mape: 10.9281
Epoch 70/200
11/11 [==============================] - 0s 2ms/step - loss: 0.2359 - mse: 0.0982 - mae: 0.2359 - mape: 6.9797
Epoch 71/200
11/11 [==============================] - 0s 2ms/step - loss: 0.3097 - mse: 0.1662 - mae: 0.3097 - mape: 12.2858
Epoch 72/200
11/11 [==============================] - 0s 2ms/step - loss: 0.3336 - mse: 0.2132 - mae: 0.3336 - mape: 12.6847
Epoch 73/200
11/11 [==============================] - 0s 2ms/step - loss: 0.2892 - mse: 0.1659 - mae: 0.2892 - mape: 11.5502
Epoch 74/200
11/

11/11 [==============================] - 0s 2ms/step - loss: 0.3149 - mse: 0.1845 - mae: 0.3149 - mape: 8.1308
Epoch 132/200
11/11 [==============================] - 0s 2ms/step - loss: 0.3711 - mse: 0.2914 - mae: 0.3711 - mape: 7.2329
Epoch 133/200
11/11 [==============================] - 0s 1ms/step - loss: 0.3343 - mse: 0.2179 - mae: 0.3343 - mape: 9.2451
Epoch 134/200
11/11 [==============================] - 0s 2ms/step - loss: 0.3057 - mse: 0.1573 - mae: 0.3057 - mape: 9.8773
Epoch 135/200
11/11 [==============================] - 0s 2ms/step - loss: 0.2407 - mse: 0.1086 - mae: 0.2407 - mape: 7.6914
Epoch 136/200
11/11 [==============================] - 0s 2ms/step - loss: 0.2325 - mse: 0.0937 - mae: 0.2325 - mape: 6.0678
Epoch 137/200
11/11 [==============================] - 0s 2ms/step - loss: 0.2383 - mse: 0.1087 - mae: 0.2383 - mape: 8.9841
Epoch 138/200
11/11 [==============================] - 0s 2ms/step - loss: 0.3056 - mse: 0.1715 - mae: 0.3056 - mape: 8.0191
Epoch 139/200


...

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    8.5s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    8.5s finished


Epoch 1/200
24/24 [==============================] - 0s 1ms/step - loss: 26.5788 - mse: 1952.8370 - mae: 26.5788 - mape: 88.0680
Epoch 2/200
24/24 [==============================] - 0s 2ms/step - loss: 7.5818 - mse: 157.1092 - mae: 7.5818 - mape: 94.3312
Epoch 3/200
24/24 [==============================] - 0s 1ms/step - loss: 5.1578 - mse: 64.1699 - mae: 5.1578 - mape: 86.0530
Epoch 4/200
24/24 [==============================] - 0s 1ms/step - loss: 3.5671 - mse: 28.0256 - mae: 3.5671 - mape: 92.5400
Epoch 5/200
24/24 [==============================] - 0s 1ms/step - loss: 3.4629 - mse: 26.8465 - mae: 3.4629 - mape: 83.6535
Epoch 6/200
24/24 [==============================] - 0s 2ms/step - loss: 3.1360 - mse: 29.9908 - mae: 3.1360 - mape: 72.1127
Epoch 7/200
24/24 [==============================] - 0s 1ms/step - loss: 2.2885 - mse: 11.5347 - mae: 2.2885 - mape: 57.8631
Epoch 8/200
24/24 [==============================] - 0s 2ms/step - loss: 2.2767 - mse: 11.5567 - mae: 2.2767 - mape: 55.

24/24 [==============================] - 0s 2ms/step - loss: 0.9230 - mse: 2.4000 - mae: 0.9230 - mape: 6.0780
Epoch 67/200
24/24 [==============================] - 0s 2ms/step - loss: 0.7872 - mse: 2.3527 - mae: 0.7872 - mape: 5.6703
Epoch 68/200
24/24 [==============================] - 0s 2ms/step - loss: 1.0389 - mse: 3.8505 - mae: 1.0389 - mape: 8.8277
Epoch 69/200
24/24 [==============================] - 0s 2ms/step - loss: 0.9427 - mse: 2.4799 - mae: 0.9427 - mape: 10.7530
Epoch 70/200
24/24 [==============================] - 0s 2ms/step - loss: 0.8700 - mse: 1.9389 - mae: 0.8700 - mape: 6.3640
Epoch 71/200
24/24 [==============================] - 0s 2ms/step - loss: 0.9138 - mse: 2.9038 - mae: 0.9138 - mape: 7.1202
Epoch 72/200
24/24 [==============================] - 0s 2ms/step - loss: 0.6459 - mse: 1.1077 - mae: 0.6459 - mape: 4.7401
Epoch 73/200
24/24 [==============================] - 0s 2ms/step - loss: 0.6368 - mse: 1.0437 - mae: 0.6368 - mape: 7.5797
Epoch 74/200
24/24 [

24/24 [==============================] - 0s 1ms/step - loss: 0.5242 - mse: 0.7664 - mae: 0.5242 - mape: 6.6691
Epoch 132/200
24/24 [==============================] - 0s 2ms/step - loss: 0.7565 - mse: 2.3700 - mae: 0.7565 - mape: 9.4822
Epoch 133/200
24/24 [==============================] - 0s 2ms/step - loss: 0.4416 - mse: 0.4804 - mae: 0.4416 - mape: 7.6347
Epoch 134/200
24/24 [==============================] - 0s 2ms/step - loss: 1.9585 - mse: 31.0195 - mae: 1.9585 - mape: 8.2247
Epoch 135/200
24/24 [==============================] - 0s 1ms/step - loss: 1.7111 - mse: 13.4933 - mae: 1.7111 - mape: 8.2932
Epoch 136/200
24/24 [==============================] - 0s 1ms/step - loss: 0.4929 - mse: 0.6425 - mae: 0.4929 - mape: 5.7151
Epoch 137/200
24/24 [==============================] - 0s 2ms/step - loss: 0.6527 - mse: 1.0282 - mae: 0.6527 - mape: 5.2180
Epoch 138/200
24/24 [==============================] - 0s 2ms/step - loss: 0.4665 - mse: 0.5286 - mae: 0.4665 - mape: 5.7466
Epoch 139/20

24/24 [==============================] - 0s 1ms/step - loss: 0.6873 - mse: 1.7514 - mae: 0.6873 - mape: 4.8699
Epoch 197/200
24/24 [==============================] - 0s 2ms/step - loss: 0.5220 - mse: 0.9742 - mae: 0.5220 - mape: 8.8627
Epoch 198/200
24/24 [==============================] - 0s 2ms/step - loss: 0.6707 - mse: 1.3351 - mae: 0.6707 - mape: 7.9622
Epoch 199/200
24/24 [==============================] - 0s 1ms/step - loss: 0.5700 - mse: 0.9025 - mae: 0.5700 - mape: 5.7092
Epoch 200/200
4/4 [==============================] - 0s 1ms/step
 
 
 
----------------------------------------------------
Feature Generation (Learning Phase): Score Generated
----------------------------------------------------
 
 
 


In [15]:
# Time-Elapsed Training on Each Part
Architope_partition_training = time.time() - Architope_partition_training_begin

---

### Train Classifier

#### Deep Classifier
Prepare Labels/Classes

In [16]:
# Time-Elapsed Training Deep Classifier
Architope_deep_classifier_training_begin = time.time()

In [17]:
# Initialize Classes Labels
partition_labels_training_integers = np.argmin(training_quality,axis=-1)
partition_labels_training = pd.DataFrame(pd.DataFrame(partition_labels_training_integers) == 0)
# Build Classes
for part_column_i in range(1,(training_quality.shape[1])):
    partition_labels_training = pd.concat([partition_labels_training,
                                           (pd.DataFrame(partition_labels_training_integers) == part_column_i)
                                          ],axis=1)
# Convert to integers
partition_labels_training = partition_labels_training+0

Re-Load Grid and Redefine Relevant Input/Output dimensions in dictionary.

In [18]:
# Re-Load Hyper-parameter Grid
exec(open('Grid_Enhanced_Network.py').read())
# Re-Load Helper Function(s)
exec(open('Helper_Functions.py').read())

# Redefine (Dimension-related) Elements of Grid
param_grid_Deep_Classifier['input_dim'] = [X_train.shape[1]]
param_grid_Deep_Classifier['output_dim'] = [partition_labels_training.shape[1]]

Deep Feature Builder - Ready
Deep Classifier - Ready


#### Train Deep Classifier

In [19]:
# Train simple deep classifier
predicted_classes_train, predicted_classes_test, N_params_deep_classifier = build_simple_deep_classifier(n_folds = CV_folds, 
                                                                                                        n_jobs = n_jobs, 
                                                                                                        n_iter =n_iter, 
                                                                                                        param_grid_in=param_grid_Deep_Classifier, 
                                                                                                        X_train = X_train, 
                                                                                                        y_train = partition_labels_training,
                                                                                                        X_test = X_test)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Epoch 1/50

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    2.7s finished


34/34 [==============================] - 0s 2ms/step - loss: 1.0759 - accuracy: 0.4712
Epoch 2/50
34/34 [==============================] - 0s 1ms/step - loss: 0.8033 - accuracy: 0.5473
Epoch 3/50
34/34 [==============================] - 0s 1ms/step - loss: 0.6825 - accuracy: 0.6160
Epoch 4/50
34/34 [==============================] - 0s 2ms/step - loss: 0.6243 - accuracy: 0.6698
Epoch 5/50
34/34 [==============================] - 0s 2ms/step - loss: 0.5862 - accuracy: 0.7161
Epoch 6/50
34/34 [==============================] - 0s 2ms/step - loss: 0.5582 - accuracy: 0.7310
Epoch 7/50
34/34 [==============================] - 0s 2ms/step - loss: 0.5280 - accuracy: 0.7644
Epoch 8/50
34/34 [==============================] - 0s 2ms/step - loss: 0.5028 - accuracy: 0.7792
Epoch 9/50
34/34 [==============================] - 0s 2ms/step - loss: 0.4829 - accuracy: 0.7904
Epoch 10/50
34/34 [==============================] - 0s 2ms/step - loss: 0.4591 - accuracy: 0.8033
Epoch 11/50
34/34 [===========

In [20]:
# Time-Elapsed Training Deep Classifier
Architope_deep_classifier_training = time.time() - Architope_deep_classifier_training_begin

Make Prediction(s)

In [21]:
# Training Set
Architope_prediction_y_train = np.take_along_axis(predictions_train, predicted_classes_train[:,None], axis=1)
# Testing Set
Architope_prediction_y_test = np.take_along_axis(predictions_test, predicted_classes_test[:,None], axis=1)

#### Write Predictions

Compute Performance

In [22]:
# Compute Peformance
performance_Architope = reporter(y_train_hat_in=Architope_prediction_y_train,
                                    y_test_hat_in=Architope_prediction_y_test,
                                    y_train_in=y_train,
                                    y_test_in=y_test)
# Write Performance
performance_Architope.to_latex((results_tables_path+"Architopes_full_performance.tex"))

# Update User
print(performance_Architope)

         train       test
MAE   0.703017   3.631131
MSE   1.588457  26.150923
MAPE  7.282801  81.668825


---

---

---

### Model Complexity/Efficiency Metrics

In [23]:
# Compute Parameters for composite models #
#-----------------------------------------#
N_params_Architope_full = N_params_Architope + N_params_deep_classifier

# Build AIC-like Metric #
#-----------------------#
AIC_like = 2*(N_params_Architope_full - np.log((performance_Architope['test']['MAE'])))
AIC_like = np.round(AIC_like,3)
Efficiency = np.log(N_params_Architope_full) *(performance_Architope['test']['MAE'])
Efficiency = np.round(Efficiency,3)


# Build Table #
#-------------#
Architope_Model_Complexity_full = pd.DataFrame({'L-time': [Architope_partition_training],
                                                  'P-time':[Architope_partitioning_max_time_running],
                                                  'N_params_expt': [N_params_Architope_full],
                                                  'AIC-like': [AIC_like],
                                                  'Eff': [Efficiency]})


# Write Required Training Time(s)
Architope_Model_Complexity_full.to_latex((results_tables_path+"Architope_full_model_complexities.tex"))

#--------------======---------------#
# Display Required Training Time(s) #
#--------------======---------------#
print(Architope_Model_Complexity_full)

      L-time     P-time  N_params_expt    AIC-like     Eff
0  32.898192  18.757979          56712  113421.421  39.745


---
---
# Benchmark(s)
---
---

### Architope with Logistic-Classifier Partitioning
#### Train Logistic Classifier (Benchmark)

In [24]:
# Time-Elapsed Training linear classifier
Architope_logistic_classifier_training_begin = time.time()

In [25]:
parameters = {'penalty': ['none','l1', 'l2'], 'C': [0.1, 0.5, 1.0, 10, 100, 1000]}
lr = LogisticRegression(random_state=2020)
cv = RepeatedStratifiedKFold(n_splits=CV_folds, n_repeats=n_iter, random_state=0)
classifier = RandomizedSearchCV(lr, parameters, random_state=2020)

# Initialize Classes Labels
partition_labels_training = np.argmin(training_quality,axis=-1)

#### Train Logistic Classifier

In [26]:
# Update User on shape of learned partition
print(partition_labels_training)

[0 1 0 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 0 1 1
 0 1 1 0 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1
 1 0 1 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 0 1 1 1 1 0 0 0 0 0
 1 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 0 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1
 1 1 0 1 0 0 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 0 1 0 1
 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 0 0 1 0 0 0 0 1 1 1 0 1 1 0 1
 1 1 0 0 1 0 1 0 1 1 1 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1
 1 0 1 0 1 0 1 0 1 0 0 0 1 1 1 1 1 0 0 1 1 1 0 0 1 1 0 0 0 1 0 0 1 1 1 0 1
 0 1 1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1
 0 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 0 0 0 1 0 1 0 1 1 1 1
 0 1 0 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 0 0 1 1 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0
 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 0 1 0 1 0 1 1 0 0 0 1 1 1 0 1 0 1 1 0 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 0 1 0 1 1 1 1 0 

In [27]:
# Update User #
#-------------#
print("Training classifier and generating partition!")

# Train Logistic Classifier #
#---------------------------#
# Supress warnings caused by "ignoring C" for 'none' penalty and similar obvious warnings
warnings.simplefilter("ignore")
# Train Classifier
classifier.fit(X_train, partition_labels_training)

Training classifier and generating partition!


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='auto', n_jobs=None,
                                                penalty='l2', random_state=2020,
                                                solver='lbfgs', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'C': [0.1, 0.5, 1.0, 10, 100, 1000],
                                        'penalty': ['none', 'l1', 'l2']},
                   pre_dispatch='2*n_jobs', random_state=2020, refit=True,
                   return_train_score=False, scoring

#### Write Predicted Class(es)

In [28]:
# Training Set
predicted_classes_train_logistic_BM = classifier.best_estimator_.predict(X_train)
Architope_prediction_y_train_logistic_BM = np.take_along_axis(predictions_train, predicted_classes_train_logistic_BM[:,None], axis=1)

# Testing Set
predicted_classes_test_logistic_BM = classifier.best_estimator_.predict(X_test)
Architope_prediction_y_test_logistic_BM = np.take_along_axis(predictions_test, predicted_classes_test_logistic_BM[:,None], axis=1)

# Extract Number of Parameters Logistic Regressor
N_params_best_logistic = (classifier.best_estimator_.coef_.shape[0])*(classifier.best_estimator_.coef_.shape[1]) + len(classifier.best_estimator_.intercept_)

In [29]:
# Time-Elapsed Training linear classifier
Architope_logistic_classifier_training = time.time() - Architope_logistic_classifier_training_begin

#### Compute Performance

In [30]:
# Compute Peformance
performance_architope_ffNN_logistic = reporter(y_train_hat_in=Architope_prediction_y_train_logistic_BM,
                                    y_test_hat_in=Architope_prediction_y_test_logistic_BM,
                                    y_train_in=y_train,
                                    y_test_in=y_test)
# Write Performance
performance_architope_ffNN_logistic.to_latex((results_tables_path+"Architopes_logistic_performance.tex"))

# Update User
print(performance_architope_ffNN_logistic)

          train       test
MAE    0.711305   3.885554
MSE    1.416262  27.732233
MAPE  10.640273  84.597842


---
## Bagged Feed-Forward Networks (ffNNs)

In [31]:
# Time for Bagging
Bagging_ffNN_bagging_time_begin = time.time()

In [32]:
# Train Bagging Weights in-sample
bagging_coefficients = LinearRegression().fit(predictions_train,y_train)

# Predict Bagging Weights out-of-sample
bagged_prediction_train = bagging_coefficients.predict(predictions_train)
bagged_prediction_test = bagging_coefficients.predict(predictions_test)

# Write number of trainable bagging parameters
N_bagged_parameters = len(bagging_coefficients.coef_) + 1

In [33]:
# Time for Bagging
Bagging_ffNN_bagging_time = time.time() - Bagging_ffNN_bagging_time_begin

In [34]:
# Compute Peformance
performance_bagged_ffNN = reporter(y_train_hat_in=bagged_prediction_train,
                                    y_test_hat_in=bagged_prediction_test,
                                    y_train_in=y_train,
                                    y_test_in=y_test)
# Write Performance
performance_bagged_ffNN.to_latex((results_tables_path+"ffNN_Bagged.tex"))

# Update User
print("Written Bagged Performance")
print(performance_bagged_ffNN)

Written Bagged Performance
           train       test
MAE     0.759085   3.092605
MSE     1.339658  18.547324
MAPE  143.040009  47.131520


In [35]:
print("Random Partition: Generated!...Feature Generation Complete!")

Random Partition: Generated!...Feature Generation Complete!


## Vanilla ffNN
#### Reload Hyper-parameter Grid

In [36]:
# Re-Load Hyper-parameter Grid
exec(open('Grid_Enhanced_Network.py').read())
# Re-Load Helper Function(s)
exec(open('Helper_Functions.py').read())
# Update Dimensions
param_grid_Vanilla_Nets['input_dim'] = [X_train.shape[1]]

Deep Feature Builder - Ready
Deep Classifier - Ready


In [37]:
# Time for Bagging
Vanilla_ffNN_time_beginn = time.time()

In [38]:
#X_train vanilla ffNNs
y_hat_train_Vanilla_ffNN, y_hat_test_Vanilla_ffNN, N_params_Vanilla_ffNN = build_ffNN(n_folds = CV_folds_failsafe, 
                                                                                   n_jobs = n_jobs, 
                                                                                   n_iter = n_iter, 
                                                                                   param_grid_in = param_grid_Vanilla_Nets, 
                                                                                   X_train=X_train, 
                                                                                   y_train=data_y, 
                                                                                   X_test_partial=X_train,
                                                                                   X_test=X_test)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    8.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    8.3s finished


Epoch 1/200
34/34 [==============================] - 0s 2ms/step - loss: 16.6619 - mse: 902.7366 - mae: 16.6619 - mape: 95.9660
Epoch 2/200
34/34 [==============================] - 0s 2ms/step - loss: 4.3196 - mse: 38.2548 - mae: 4.3196 - mape: 124.5451
Epoch 3/200
34/34 [==============================] - 0s 2ms/step - loss: 2.9448 - mse: 17.1179 - mae: 2.9448 - mape: 95.5345
Epoch 4/200
34/34 [==============================] - 0s 2ms/step - loss: 2.4650 - mse: 13.8265 - mae: 2.4650 - mape: 76.2711
Epoch 5/200
34/34 [==============================] - 0s 2ms/step - loss: 2.4883 - mse: 16.0393 - mae: 2.4883 - mape: 76.5220
Epoch 6/200
34/34 [==============================] - 0s 2ms/step - loss: 2.2012 - mse: 12.3020 - mae: 2.2012 - mape: 66.3522
Epoch 7/200
34/34 [==============================] - 0s 2ms/step - loss: 1.6369 - mse: 6.7081 - mae: 1.6369 - mape: 53.0985
Epoch 8/200
34/34 [==============================] - 0s 2ms/step - loss: 1.7808 - mse: 8.8747 - mae: 1.7808 - mape: 40.748

34/34 [==============================] - 0s 2ms/step - loss: 0.6709 - mse: 1.2663 - mae: 0.6709 - mape: 7.1128
Epoch 67/200
34/34 [==============================] - 0s 2ms/step - loss: 0.9811 - mse: 3.0402 - mae: 0.9811 - mape: 9.8283
Epoch 68/200
34/34 [==============================] - 0s 2ms/step - loss: 0.7194 - mse: 1.3852 - mae: 0.7194 - mape: 7.8421
Epoch 69/200
34/34 [==============================] - 0s 2ms/step - loss: 0.6880 - mse: 1.8176 - mae: 0.6880 - mape: 10.1175
Epoch 70/200
34/34 [==============================] - 0s 2ms/step - loss: 0.6288 - mse: 1.3551 - mae: 0.6288 - mape: 6.5860
Epoch 71/200
34/34 [==============================] - 0s 2ms/step - loss: 0.7192 - mse: 1.8498 - mae: 0.7192 - mape: 8.3804
Epoch 72/200
34/34 [==============================] - 0s 2ms/step - loss: 0.5224 - mse: 1.1818 - mae: 0.5224 - mape: 6.6911
Epoch 73/200
34/34 [==============================] - 0s 1ms/step - loss: 0.7934 - mse: 2.3638 - mae: 0.7934 - mape: 8.3734
Epoch 74/200
34/34 [

34/34 [==============================] - 0s 1ms/step - loss: 0.5592 - mse: 1.0882 - mae: 0.5592 - mape: 5.7931
Epoch 132/200
34/34 [==============================] - 0s 2ms/step - loss: 0.5183 - mse: 0.7616 - mae: 0.5183 - mape: 6.7383
Epoch 133/200
34/34 [==============================] - 0s 1ms/step - loss: 0.4590 - mse: 0.6282 - mae: 0.4590 - mape: 5.5308
Epoch 134/200
34/34 [==============================] - 0s 1ms/step - loss: 0.4999 - mse: 0.8267 - mae: 0.4999 - mape: 6.1082
Epoch 135/200
34/34 [==============================] - 0s 1ms/step - loss: 0.7544 - mse: 2.3511 - mae: 0.7544 - mape: 6.6611
Epoch 136/200
34/34 [==============================] - 0s 1ms/step - loss: 0.4877 - mse: 1.0408 - mae: 0.4877 - mape: 7.8620
Epoch 137/200
34/34 [==============================] - 0s 1ms/step - loss: 0.4068 - mse: 0.4919 - mae: 0.4068 - mape: 6.3422
Epoch 138/200
34/34 [==============================] - 0s 1ms/step - loss: 0.3556 - mse: 0.3702 - mae: 0.3556 - mape: 4.3772
Epoch 139/200


34/34 [==============================] - 0s 2ms/step - loss: 0.4611 - mse: 0.8388 - mae: 0.4611 - mape: 5.3030
Epoch 197/200
34/34 [==============================] - 0s 2ms/step - loss: 0.5570 - mse: 1.7331 - mae: 0.5570 - mape: 5.4423
Epoch 198/200
34/34 [==============================] - 0s 1ms/step - loss: 0.5126 - mse: 0.9307 - mae: 0.5126 - mape: 5.3564
Epoch 199/200
34/34 [==============================] - 0s 2ms/step - loss: 0.5605 - mse: 1.0170 - mae: 0.5605 - mape: 4.3216
Epoch 200/200
4/4 [==============================] - 0s 1ms/step


In [39]:
# Time for Bagging
Vanilla_ffNN_time = time.time() - Vanilla_ffNN_time_beginn

In [40]:
# Update User #
#-------------#
print("Trained vanilla ffNNs")

Trained vanilla ffNNs


#### Evaluate Performance

In [41]:
# Compute Peformance
performance_Vanilla_ffNN = reporter(y_train_hat_in=y_hat_train_Vanilla_ffNN,y_test_hat_in=y_hat_test_Vanilla_ffNN,y_train_in=y_train,y_test_in=y_test)
# Write Performance
performance_Vanilla_ffNN.to_latex((results_tables_path+"ffNN_Vanilla.tex"))

# Update User #
#-------------#
print("Written Bagged Vanilla ffNNs")
print(performance_Vanilla_ffNN)

Written Bagged Vanilla ffNNs
          train       test
MAE    0.877264   3.057480
MSE    2.589455  15.766277
MAPE  10.028579  29.520332


#### Compute Required Training Time(s)

In [42]:
# In-Line #
#---------#

# Architope (Full) Time Lapse
Architope_Full_Time = partitioning_time + Architope_partition_training + Architope_deep_classifier_training
# Architope (Logistic) Time Lapse
Architope_logistic_Time = partitioning_time + Architope_partition_training + Architope_logistic_classifier_training
# Bagged ffNN Training Time
Bagged_ffNN_Time = partitioning_time + Architope_partition_training + Bagging_ffNN_bagging_time
# Vanilla ffNN
Vanilla_ffNN_Time = Vanilla_ffNN_time

# Parallel (Only if applicable) #
#-------------------------------#

# Architope (Full) Time Lapse
Architope_Full_Time_parallel = partitioning_time + Architope_partitioning_max_time_running + Architope_deep_classifier_training
# Architope (Logistic) Time Lapse
Architope_logistic_Time_parallel = partitioning_time + Architope_partitioning_max_time_running + Architope_logistic_classifier_training
# Bagged ffNN Training Time
Bagged_ffNN_Time_parallel = partitioning_time + Architope_partitioning_max_time_running + Bagging_ffNN_bagging_time

#### Write Required Training Times

In [43]:
# Update User #
#-------------#
print('Writing preliminary table: Required Training Times')

# Format Required Training Time(s)
training_times_In_Line = pd.DataFrame({'Architope': [round(Architope_Full_Time,3)],
                                'Architope-logistic': [round(Architope_logistic_Time,3)],
                                'Vanilla ffNN': [round(Vanilla_ffNN_Time,3)],
                                'Bagged ffNN': [round(Bagged_ffNN_Time,3)]})
training_times_Parallel = pd.DataFrame({'Architope': [round(Architope_Full_Time_parallel,3)],
                                'Architope-logistic': [round(Architope_logistic_Time_parallel,3)],
                                'Vanilla ffNN': ['-'],
                                'Bagged ffNN': [round(Bagged_ffNN_Time_parallel,3)]})

# Combine Training Times into Single Data-Frame #
#-----------------------------------------------#
Model_Training_times = training_times_In_Line.append(training_times_Parallel)

# Write Required Training Time(s)
Model_Training_times.to_latex((results_tables_path+"Model_Training_Times.tex"))
# Display Required Training Time(s)
print(Model_Training_times)

Writing preliminary table: Required Training Times
   Architope  Architope-logistic Vanilla ffNN  Bagged ffNN
0     45.094              42.091       22.285       38.261
0     30.954              27.951            -       24.121


## Run: Gradient Boosted Random Forest Regression

In [44]:
# Update User #
#-------------#
print('Training Gradient-Boosted Random Forest: In-progress...')
# Run from External Script
exec(open('Gradient_Boosted_Random_Forest_Regressor.py').read())

# Update User #
#-------------#
print('Training of Gradient-Boosted Random Forest: Complete!')

Training Gradient-Boosted Random Forest: In-progress...
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.4s finished


Gradient Boosted Trees Model - Done CV!
Random Forest Regressor uses: 34580 parameters.
           train        test
MAE    15.041325   19.473950
MSE   523.639972  626.541744
MAPE  298.530655  528.589826
Training of Gradient-Boosted Random Forest: Complete!


## Training Result(s)
#### (Update) Write Required Training Times

In [45]:
# Update User #
#-------------#
print('Completing Table: Required Training Times')

# Format Required Training Time(s)
training_times_In_Line = pd.DataFrame({'Vanilla ffNN': [round(Vanilla_ffNN_Time,3)],
                                       'Grad.Bstd Rand.F': [round(Gradient_boosted_Random_forest_time,3)],
                                       'Bagged ffNN': [round(Bagged_ffNN_Time,3)],
                                       'Architope-logistic': [round(Architope_logistic_Time,3)],
                                       'Architope': [round(Architope_Full_Time,3)]
                                      },index=['In-Line (L-Time)'])

training_times_Parallel = pd.DataFrame({'Vanilla ffNN': ['-'],
                                        'Grad.Bstd Rand.F': ['-'],
                                        'Bagged ffNN': [round(Bagged_ffNN_Time_parallel,3)],
                                        'Architope-logistic': [round(Architope_logistic_Time_parallel,3)],
                                        'Architope': [round(Architope_Full_Time_parallel,3)]},index=['Parallel (P-Time)'])

# Combine Training Times into Single Data-Frame #
#-----------------------------------------------#
Model_Training_times = training_times_In_Line.append(training_times_Parallel)
Model_Training_times = Model_Training_times.transpose()

# Write Required Training Time(s)
Model_Training_times.to_latex((results_tables_path+"Model_Training_Times.tex"))
# Display Required Training Time(s)
print(Model_Training_times)

Completing Table: Required Training Times
                   In-Line (L-Time) Parallel (P-Time)
Vanilla ffNN                 22.285                 -
Grad.Bstd Rand.F              0.934                 -
Bagged ffNN                  38.261            24.121
Architope-logistic           42.091            27.951
Architope                    45.094            30.954


### Prediction Metric(s)
#### Write Predictive Performance Dataframe(s)

In [46]:
# Write Training Performance
predictive_performance_training = pd.DataFrame({'ffNN': performance_Vanilla_ffNN.train,
                                                'GBRF': Gradient_boosted_tree.train,
                                                'ffNN-bag': performance_bagged_ffNN.train,
                                                'ffNN-lgt': performance_architope_ffNN_logistic.train,
                                                'tope': performance_Architope.train})
predictive_performance_training = predictive_performance_training.transpose()

# Write Testing Performance
predictive_performance_test = pd.DataFrame({'ffNN': performance_Vanilla_ffNN.test,
                                            'GBRF': Gradient_boosted_tree.test,
                                            'ffNN-bag': performance_bagged_ffNN.test,
                                            'ffNN-lgt': performance_architope_ffNN_logistic.test,
                                            'tope': performance_Architope.test})
predictive_performance_test = predictive_performance_test.transpose()

# Write into one Dataframe #
#--------------------------#
predictive_performance_training.to_latex((results_tables_path+"Models_predictive_performance_training.tex"))
predictive_performance_test.to_latex((results_tables_path+"Models_predictive_performance_testing.tex"))

# Update User #
#-------------#
print(predictive_performance_training)

                MAE         MSE        MAPE
ffNN       0.877264    2.589455   10.028579
GBRF      15.041325  523.639972  298.530655
ffNN-bag   0.759085    1.339658  143.040009
ffNN-lgt   0.711305    1.416262   10.640273
tope       0.703017    1.588457    7.282801


### Model Complexity/Efficiency Metrics

In [47]:
# Compute Parameters for composite models #
#-----------------------------------------#
N_params_Architope_full = N_params_Architope + N_params_deep_classifier
N_params_Architope_logistic = N_params_Architope + N_params_best_logistic
N_params_bagged_ffNN = N_params_Architope + N_bagged_parameters

# Build Table #
#-------------#
Number_of_model_parameters = pd.DataFrame({'Vanilla ffNN': [N_params_Vanilla_ffNN],
                                           'Grad.Bstd Rand.F': [N_tot_params_in_forest],
                                           'Bagged ffNN': [N_params_bagged_ffNN],
                                           'Architope-logistic': [N_params_Architope_logistic],
                                           'Architope': [N_params_Architope_full]},
                                            index=['N_par'])

Number_of_model_parameters = Number_of_model_parameters.transpose()

# Append to Dataframe #
#---------------------#
Model_Complexity_Metrics = Model_Training_times
Model_Complexity_Metrics['N_par']=Number_of_model_parameters.values

# Build AIC-like Metric #
#-----------------------#
AIC_like = 2*((Model_Complexity_Metrics.N_par.values) - np.log(predictive_performance_test.MAE.values))
AIC_like = np.round(AIC_like,3)
Efficiency = np.log(Model_Complexity_Metrics.N_par.values) *predictive_performance_test.MAE.values
Efficiency = np.round(Efficiency,3)


# Update Training Metrics Dataframe #
#-----------------------------------#
Model_Complexity_Metrics['AIC_like'] = AIC_like
Model_Complexity_Metrics['Eff'] = Efficiency

# Write Required Training Time(s)
Model_Complexity_Metrics.to_latex((results_tables_path+"Model_Complexity_Metrics.tex"))

#--------------======---------------#
# Display Required Training Time(s) #
#--------------======---------------#
print(Model_Complexity_Metrics)

                   In-Line (L-Time) Parallel (P-Time)  N_par    AIC_like  \
Vanilla ffNN                 22.285                 -  27351   54699.765   
Grad.Bstd Rand.F              0.934                 -  34580   69154.062   
Bagged ffNN                  38.261            24.121  54705  109407.742   
Architope-logistic           42.091            27.951  55197  110391.285   
Architope                    45.094            30.954  56712  113421.421   

                        Eff  
Vanilla ffNN         31.237  
Grad.Bstd Rand.F    203.523  
Bagged ffNN          33.739  
Architope-logistic   42.425  
Architope            39.745  


# Summary

In [48]:
print(' ')
print(' ')
print('#-------------------#')
print(' PERFORMANCE SUMMARY:')
print('#-------------------#')
print(' ')
print(' ')
print('#===================#')
print(' Individual Metrics: ')
print('#======-============#')
print(' ')
print('----------------------------------------')
print('----------------------------------------')
print('Architope (Full)')
print('----------------------------------------')
print(performance_Architope)
print('----------------------------------------')
print('Architope - Naive Logistic')
print('----------------------------------------')
print(performance_architope_ffNN_logistic)
print('----------------------------------------')
print('Vanilla ffNN')
print('----------------------------------------')
print(performance_Vanilla_ffNN)
print('----------------------------------------')
print('Bagged ffNN')
print('----------------------------------------')
print(performance_bagged_ffNN)
print('----------------------------------------')
print('Gradient Boosted Random Forest Regressor')
print('----------------------------------------')
print(Gradient_boosted_tree)
print('----------------------------------------')
print('----------------------------------------')
print(' ')
print(' ')
#
print('#==================#')
print(' Overview  Metrics : ')
print('#==================#')
print(' ')
print('----------------------------------------')
print('Training Performance: ')
print('----------------------------------------')
print(predictive_performance_training)
print('----------------------------------------')
print('Testing Performance: ')
print('----------------------------------------')
print(predictive_performance_test)
print('----------------------------------------')
print(' ')
print(' ')
#
print('#====================#')
print(' Efficiency Metrics: ')
print('#====================#')
print(' ')
print('Model Training Times:')
print('----------------------------------------')
print(Model_Complexity_Metrics)
print(' ')
print(' ')
print('😃😃 Have a great day!! 😃😃 ')

 
 
#-------------------#
 PERFORMANCE SUMMARY:
#-------------------#
 
 
#===================#
 Individual Metrics: 
#======-============#
 
----------------------------------------
----------------------------------------
Architope (Full)
----------------------------------------
         train       test
MAE   0.703017   3.631131
MSE   1.588457  26.150923
MAPE  7.282801  81.668825
----------------------------------------
Architope - Naive Logistic
----------------------------------------
          train       test
MAE    0.711305   3.885554
MSE    1.416262  27.732233
MAPE  10.640273  84.597842
----------------------------------------
Vanilla ffNN
----------------------------------------
          train       test
MAE    0.877264   3.057480
MSE    2.589455  15.766277
MAPE  10.028579  29.520332
----------------------------------------
Bagged ffNN
----------------------------------------
           train       test
MAE     0.759085   3.092605
MSE     1.339658  18.547324
MAPE  143.040009

---
# Fin
---